## Project by Ahmed and Joris

### 3. Import Packages, read data, prepare data 

This contains the code for the first 4 algorithms (KNN, Logistic regression, decision tree, random forest). Another file will contain the code for the camemBERT model. Import packages and load data

In [2]:
# read data and import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style("whitegrid")

!pip install -U spacy


# import some additional packages
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn. preprocessing import StandardScaler

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

# train data
df_train = pd.read_csv('training_data.csv')

# test data
df_test = pd.read_csv('unlabelled_test_data.csv')
df_test

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


,id,sentence
0,0,Nous dûmes nous excuser des propos que nous eû...
1,1,Vous ne pouvez pas savoir le plaisir que j'ai ...
2,2,"Et, paradoxalement, boire froid n'est pas la b..."
3,3,"Ce n'est pas étonnant, car c'est une saison my..."
4,4,"Le corps de Golo lui-même, d'une essence aussi..."
...,...,...
1195,1195,C'est un phénomène qui trouve une accélération...
1196,1196,Je vais parler au serveur et voir si on peut d...
1197,1197,Il n'était pas comme tant de gens qui par pare...
1198,1198,Ils deviennent dangereux pour notre économie.


### 4. Train your models

Set your X and y variables. 
Set the `random_state=0`
Split the data into a train and test set using the following parameters `train_test_split(X, y, test_size=0.2, random_state=0)`.

#### 4.1.Baseline
What is the baseline for this classification problem?

In [3]:
# Set x and y
y = df_train[['difficulty']] # labels
x = df_train[['sentence']] #phrases

# Train test split 80/20
X_train, X_test, y_train, y_test = train_test_split(x, y, shuffle=True, test_size=0.20, random_state=0, stratify=y)

# Calculate baseline
y_counts = df_train['difficulty'].value_counts().tolist()
print(y_counts)
baserate = (max(y_counts) / len(df_train.index))*100
print(baserate)



[813, 807, 798, 795, 795, 792]
16.9375


#### 4.2. Logistic Regression (without data cleaning)

Train a simple logistic regression model using a Tfidf vectoriser.

In [4]:
# Training a simple logistic regression using a TF-idf vectoriser
# No data cleaning
from sklearn.pipeline import Pipeline
from sklearn import metrics

# Use Tfidf vectoriser
txt_Tfidf = TfidfVectorizer()
X_train_txt = txt_Tfidf.fit_transform(X_train['sentence'])
X_test_txt = txt_Tfidf.transform(X_test['sentence'])

# Train log reg
lreg = LogisticRegression(random_state=0).fit(X_train_txt, y_train)

# Make pipeline
pipe = Pipeline([('vectorizer', txt_Tfidf),
                 ('classifier', lreg)])
pipe.fit(X_train['sentence'], y_train)

# Calculate accuracy
acc_lreg = lreg.score(X_test_txt, y_test)*100
print(acc_lreg)

# Evaluate the model
def evaluate(true, pred):
    precision = precision_score(true, pred, average = 'weighted', labels=['A1', 'A2', 'B1', 'B2', 'C1', 'C2'])
    recall = recall_score(true, pred, average = 'weighted', labels=['A1', 'A2', 'B1', 'B2', 'C1', 'C2'])
    f1 = f1_score(true, pred, average = 'weighted', labels=['A1', 'A2', 'B1', 'B2', 'C1', 'C2'])
    print(f"CONFUSION MATRIX:\n{confusion_matrix(true, pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(true, pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n\tPrecision: {precision:.4f}\n\tRecall: {recall:.4f}\n\tF1_Score: {f1:.4f}")

# Predictions and evaluation
y_pred_lreg = pipe.predict(X_test['sentence'])
evaluate(y_test, y_pred_lreg)





/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


46.041666666666664
CONFUSION MATRIX:
[[104  27  17   7   5   3]
 [ 36  65  36   7   7   8]
 [ 24  37  54  16  11  17]
 [  3   4  13  66  28  44]
 [  1   5   7  34  59  54]
 [  8   6  10  14  29  94]]
ACCURACY SCORE:
0.4604
CLASSIFICATION REPORT:
	Precision: 0.4582
	Recall: 0.4604
	F1_Score: 0.4561


#### 4.3. KNN (without data cleaning)

Train a KNN classification model using a Tfidf vectoriser. Show the accuracy, precision, recall and F1 score on the test set.

In [5]:
# import some additional packages 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import neighbors

# Make knn without parameter tuning (select k=5)
knn_notTuned = KNeighborsClassifier(n_neighbors = 5, weights = "distance").fit(X_train_txt, y_train)

# Make pipeline for KNN
pipeknn = Pipeline([('vectorizer', txt_Tfidf),
                 ('classifier', knn_notTuned)])
pipeknn.fit(X_train['sentence'], y_train)

# Calculate accuracy, precision, recall, and F1 score
y_pred_knn = pipeknn.predict(X_test['sentence'])
evaluate(y_test, y_pred_knn)

/usr/local/lib/python3.8/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.8/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


CONFUSION MATRIX:
[[127  23   9   1   1   2]
 [ 73  66  16   4   0   0]
 [ 77  49  25   5   1   2]
 [ 24  31  32  44  14  13]
 [ 36  28  24  17  38  17]
 [ 30  24  24   8  17  58]]
ACCURACY SCORE:
0.3729
CLASSIFICATION REPORT:
	Precision: 0.4267
	Recall: 0.3729
	F1_Score: 0.3604


Try to improve it by tuning the hyper parameters (`n_neighbors`,   `p`, `weights`).

In [6]:
# Tune parameter for KNN using grid search
knn_tuned = KNeighborsClassifier()
k_range = list(range(1, 20))
param_grid = dict(n_neighbors=k_range)

# Use the whole dataset for tuning
X_txt = txt_Tfidf.fit_transform(x['sentence'])

grid = GridSearchCV(knn_tuned, param_grid, cv=5, scoring='accuracy', return_train_score=False, verbose=1)
grid_search=grid.fit(X_txt, y)
print(grid_search.best_params_)
print(grid_search.best_score_ )

# Based on tuning, select k=1

# Make knn WITH parameter tuning
knn_Tuned = KNeighborsClassifier(n_neighbors = 1, weights = "distance").fit(X_train_txt, y_train)

# Make pipeline for KNN
pipeknn2 = Pipeline([('vectorizer', txt_Tfidf),
                 ('classifier', knn_Tuned)])
pipeknn2.fit(X_train['sentence'], y_train)

# Calculate accuracy, precision, recall, and F1 score
y_pred_knn2 = pipeknn2.predict(X_test['sentence'])
evaluate(y_test, y_pred_knn2)

Fitting 5 folds for each of 19 candidates, totalling 95 fits


/usr/local/lib/python3.8/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.8/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.8/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.8/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for exam

{'n_neighbors': 1}
0.35333333333333333
CONFUSION MATRIX:
[[125  22  10   4   0   2]
 [ 58  65  30   3   2   1]
 [ 63  45  34   8   4   5]
 [ 29  27  32  42  14  14]
 [ 35  26  23  23  35  18]
 [ 39  18  21  10  21  52]]
ACCURACY SCORE:
0.3677
CLASSIFICATION REPORT:
	Precision: 0.3997
	Recall: 0.3677
	F1_Score: 0.3530


/usr/local/lib/python3.8/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


#### 4.4. Decision Tree Classifier (without data cleaning)

Train a Decison Tree classifier, using a Tfidf vectoriser. Show the accuracy, precision, recall and F1 score on the test set.

In [7]:
#import some additional packages 
from sklearn.tree import DecisionTreeClassifier

# Make DT classifier without parameter tuning (select k=5)
tree_notTuned = DecisionTreeClassifier(max_depth=10,random_state=0).fit(X_train_txt, y_train)

# Make pipeline for tree
pipetree = Pipeline([('vectorizer', txt_Tfidf),
                 ('classifier', tree_notTuned)])
pipetree.fit(X_train['sentence'], y_train)

# Calculate accuracy, precision, recall, and F1 score
y_pred_tree = pipetree.predict(X_test['sentence'])
evaluate(y_test, y_pred_tree)

CONFUSION MATRIX:
[[108  15  10  19   7   4]
 [ 60  26  30  23  17   3]
 [ 43  28  33  25  23   7]
 [ 22   4  14  60  42  16]
 [ 20   8  13  48  59  12]
 [ 22  15  10  39  52  23]]
ACCURACY SCORE:
0.3219
CLASSIFICATION REPORT:
	Precision: 0.3159
	Recall: 0.3219
	F1_Score: 0.3000


Try to improve it by tuning the hyper parameters (`max_depth`, the depth of the decision tree).

In [8]:
# Tune parameter for DT using grid search
tree_tuned = DecisionTreeClassifier(random_state=0)
depth_range = list(range(1, 101))
param_grid = dict(max_depth=depth_range)

# Use the whole dataset for tuning
X_txt = txt_Tfidf.fit_transform(x['sentence'])

grid = GridSearchCV(tree_tuned, param_grid, cv=5, scoring='accuracy', return_train_score=False, verbose=1)
grid_search=grid.fit(X_txt, y)
print(grid_search.best_params_)
print(grid_search.best_score_ )

# # Based on tuning, select max_depth=70

# # Make tree WITH parameter tuning
tree_Tuned = DecisionTreeClassifier(max_depth=58,random_state=0).fit(X_train_txt, y_train)

# Make pipeline for tree
pipetree2 = Pipeline([('vectorizer', txt_Tfidf),
                 ('classifier', tree_Tuned)])
pipetree2.fit(X_train['sentence'], y_train)

# Calculate accuracy, precision, recall, and F1 score
y_pred_tree2 = pipetree2.predict(X_test['sentence'])
evaluate(y_test, y_pred_tree2)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'max_depth': 58}
0.333125
CONFUSION MATRIX:
[[88 29 21 11  4 10]
 [42 52 36  9  6 14]
 [29 45 31 19 23 12]
 [13 22 23 45 26 29]
 [10 11 35 39 29 36]
 [14 15 27 29 31 45]]
ACCURACY SCORE:
0.3021
CLASSIFICATION REPORT:
	Precision: 0.2964
	Recall: 0.3021
	F1_Score: 0.2975


#### 4.5. Random Forest Classifier (without data cleaning)

Try a Random Forest Classifier, using a Tfidf vectoriser. Show the accuracy, precision, recall and F1 score on the test set.

In [7]:
#import some additional packages 
from sklearn.ensemble import RandomForestClassifier
# rf_tuned = RandomForestClassifier(random_state=0)
# param_grid = { 'max_depth':[10, 20, 40, 60, 80, 100, 120, 150],  'n_estimators': [100, 200, 400, 500,  600, 800, 1000, 1500]}


# # Use the whole dataset for tuning
# X_txt = txt_Tfidf.fit_transform(x['sentence'])

# grid = GridSearchCV(rf_tuned, param_grid, cv=3, scoring='accuracy', return_train_score=False, verbose=1)
# grid_search=grid.fit(X_txt, y)
# print(grid_search.best_params_)
# print(grid_search.best_score_ )


# Make 'RF' classifier 
rf = RandomForestClassifier(n_estimators = 500, max_depth=80,random_state=0).fit(X_train_txt, y_train)

# Make pipeline for tree
piperf = Pipeline([('vectorizer', txt_Tfidf),
                 ('classifier', rf)])
piperf.fit(X_train['sentence'], y_train)

# Calculate accuracy, precision, recall, and F1 score
y_pred_rf = piperf.predict(X_test['sentence'])
evaluate(y_test, y_pred_rf)


<ipython-input-7-8dcf78643f6f>:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf = RandomForestClassifier(n_estimators = 500, max_depth=80,random_state=0).fit(X_train_txt, y_train)
/usr/local/lib/python3.8/dist-packages/sklearn/pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


CONFUSION MATRIX:
[[127  20  12   1   2   1]
 [ 66  52  25  10   3   3]
 [ 36  45  53  15   9   1]
 [  9  14  24  59  30  22]
 [ 14   7  19  47  46  27]
 [ 18  11  17  25  23  67]]
ACCURACY SCORE:
0.4208
CLASSIFICATION REPORT:
	Precision: 0.4187
	Recall: 0.4208
	F1_Score: 0.4099
